# 👩🏽‍💻 1. Prep

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier

In [ ]:
import modelbit
mb = modelbit.login(branch="development")

# 🧠 2. Training


In [ ]:
transactions = mb.get_dataset("mobile_transactions")
transactions

In [ ]:
def make_df_for_inference(input_df: pd.DataFrame) -> pd.DataFrame:
    inference_df = pd.DataFrame()
    inference_df["TYPE"] = input_df["TYPE"]
    inference_df["AMOUNT"] = input_df["AMOUNT"]
    inference_df["DESTINATION_DELTA"] = input_df["NEW_BALANCE_DESTINATION"] - input_df["OLD_BALANCE_DESTINATION"]
    return inference_df

X = make_df_for_inference(transactions)
X

In [ ]:
y = transactions["IS_FRAUD"].astype(int)
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
pipeline = Pipeline([
    ("encoder", OneHotEncoder(handle_unknown='ignore')),
    ("classifier", XGBClassifier())
])
pipeline.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, pipeline.predict(X_test))

# 🚀 3. Deployment

In [ ]:
def fraud_probability(input_df: pd.DataFrame) -> np.ndarray:
    inference_df = make_df_for_inference(input_df)
    return pipeline.predict(inference_df)

test_df = pd.DataFrame({
    "TYPE": ["CASH_OUT"],
    "AMOUNT": [50.0],
    "NEW_BALANCE_DESTINATION": [100.0],
    "OLD_BALANCE_DESTINATION": [150.0]
})
fraud_probability(test_df)

In [ ]:
%%time
fraud_probability(test_df)

In [ ]:
mb.deploy(fraud_probability, dataframe_mode=True, example_dataframe=test_df)